In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from pathlib import Path
from clean.Cleaning import data_check
from clean.Cleaning import idcheck
from clean.Cleaning import record
import os

In [3]:
df_c = pd.read_csv(r'data/RAW/clients.csv', encoding="UTF8")
df_v = pd.read_csv(r'data/RAW/ventes.csv', encoding="UTF8")
df_p = pd.read_csv(r'data/RAW/produits.csv', encoding="UTF8")

### Client 

In [45]:
df_c.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8623 entries, 0 to 8622
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   client_id  8623 non-null   object
 1   sex        8623 non-null   object
 2   birth      8623 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 202.2+ KB


In [5]:
df_c.describe(include ='all') 

,client_id,sex,birth
count,8623,8623,8623.000000
unique,8623,2,NaN
top,c_2990,f,NaN
freq,1,4491,NaN
mean,NaN,NaN,1978.280877
std,NaN,NaN,16.919535
min,NaN,NaN,1929.000000
25%,NaN,NaN,1966.000000
50%,NaN,NaN,1979.000000
75%,NaN,NaN,1992.000000


In [6]:
df_c[~df_c.client_id.str.contains("c_")]

,client_id,sex,birth
2735,ct_0,f,2001
8494,ct_1,m,2001


-> we have spotted 2 client_id  which does not have the same format as most 

### Produit

In [7]:
df_p.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3287 entries, 0 to 3286
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id_prod  3287 non-null   object 
 1   price    3287 non-null   float64
 2   categ    3287 non-null   int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 77.2+ KB


In [8]:
df_p.describe(include='all')

,id_prod,price,categ
count,3287,3287.000000,3287.000000
unique,3287,NaN,NaN
top,0_1205,NaN,NaN
freq,1,NaN,NaN
mean,NaN,21.856641,0.370246
std,NaN,29.847908,0.615387
min,NaN,-1.000000,0.000000
25%,NaN,6.990000,0.000000
50%,NaN,13.060000,0.000000
75%,NaN,22.990000,1.000000


->  we have spotted a negative price -1

In [9]:
df_p[df_p['price']<= 0]

,id_prod,price,categ
731,T_0,-1.0,0


### ventes

In [46]:
df_v.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 337016 entries, 0 to 337015
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   id_prod     337016 non-null  object
 1   date        337016 non-null  object
 2   session_id  337016 non-null  object
 3   client_id   337016 non-null  object
dtypes: object(4)
memory usage: 10.3+ MB


In [11]:
df_v.describe()

,id_prod,date,session_id,client_id
count,337016,337016,337016,337016
unique,3266,336855,169195,8602
top,1_369,test_2021-03-01 02:30:02.237413,s_0,c_1609
freq,1081,13,200,12855


 -> we have spotted a date that contains " test_" and date type is object instead datetime
 
 -> 12855 as frequency for "c_1606" client's Id  

In [12]:
df_v[df_v.date.str.contains("test")]

,id_prod,date,session_id,client_id
1431,T_0,test_2021-03-01 02:30:02.237420,s_0,ct_1
2365,T_0,test_2021-03-01 02:30:02.237446,s_0,ct_1
2895,T_0,test_2021-03-01 02:30:02.237414,s_0,ct_1
5955,T_0,test_2021-03-01 02:30:02.237441,s_0,ct_0
7283,T_0,test_2021-03-01 02:30:02.237434,s_0,ct_1
...,...,...,...,...
332594,T_0,test_2021-03-01 02:30:02.237445,s_0,ct_0
332705,T_0,test_2021-03-01 02:30:02.237423,s_0,ct_1
332730,T_0,test_2021-03-01 02:30:02.237421,s_0,ct_1
333442,T_0,test_2021-03-01 02:30:02.237431,s_0,ct_1


-> after having explored the different tables we have located data intended for 'tests', we will eliminate them

In [53]:
clients = df_c[~df_c.client_id.str.contains("ct")].reset_index(drop=True)
produits = df_p[~df_p.id_prod.str.contains("T_0")].reset_index(drop=True)
ventes=df_v[~df_v.date.str.contains("test")].reset_index(drop=True)

### consistency check before merge

In [14]:
data_check(clients)

 This Df contains 0 NAN, 0 NULL and 0 dublicate line


In [15]:
data_check(produits)

 This Df contains 0 NAN, 0 NULL and 0 dublicate line


In [16]:
data_check(ventes)

 This Df contains 0 NAN, 0 NULL and 0 dublicate line


In [17]:
idcheck(clients,clients.client_id, ventes.client_id)

There is 21 line of first  dataframe missing in second  dataframe


In [18]:
idcheck(produits, produits.id_prod, ventes.id_prod)

There is 22 line of first  dataframe missing in second  dataframe


In [19]:
idcheck(ventes, ventes.id_prod, produits.id_prod)

There is 103 line of first  dataframe missing in second  dataframe


In [20]:
idcheck(ventes, ventes.client_id, clients.client_id)

There is 0 line of first  dataframe missing in second  dataframe


In [63]:
Df4 = ventes.assign(result =ventes['id_prod'].isin(produits['id_prod']).astype(int))
Df4[Df4['result']!=1].groupby('id_prod').count()

,date,session_id,client_id,result
id_prod,,,,
0_2245,103,103,103,103


In [64]:
prod_missing=ventes[ventes.id_prod.str.contains("0_2245")]

,id_prod,date,session_id,client_id
6231,0_2245,2021-06-17 03:03:12.668129,s_49705,c_1533
10797,0_2245,2021-06-16 05:53:01.627491,s_49323,c_7954
14045,0_2245,2021-11-24 17:35:59.911427,s_124474,c_5120
17480,0_2245,2022-02-28 18:08:49.875709,s_172304,c_4964
21071,0_2245,2021-03-01 00:09:29.301897,s_3,c_580
...,...,...,...,...
322523,0_2245,2021-04-06 19:59:19.462288,s_16936,c_4167
329226,0_2245,2021-03-30 23:29:02.347672,s_13738,c_7790
330297,0_2245,2021-12-03 14:14:40.444177,s_128815,c_6189
335331,0_2245,2021-04-27 18:58:47.703374,s_26624,c_1595


In [65]:
ventes=ventes[~ventes.id_prod.str.contains("0_2245")]

#### -> we have some values in some table that do not match in another

In [66]:
fusion = ventes.merge(produits, how='left', left_on = 'id_prod', right_on='id_prod' )
df = fusion.merge(clients, how='left', left_on = 'client_id', right_on='client_id' )

In [67]:
data_check(df)

 This Df contains 0 NAN, 0 NULL and 0 dublicate line


#####  -> date contents check

In [26]:
df['date'] = df['date'].astype('datetime64')
print(sorted(df['date'].dt.day.unique()))
print('')
print(sorted(df['date'].dt.month.unique()))
print('')
print(sorted(df['date'].dt.year.unique()))

In [70]:
record(df,'data_clean','CURATED')
record(produits,'produit_clean','CURATED')
record(clients,'client_clean','CURATED')
